In [1]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:

!pip install -qq transformers
!pip install -qq optuna
!pip install -qq datasets

In [3]:
import transformers
import datasets
from transformers import AutoTokenizer,AutoModelForQuestionAnswering, AutoModelForSequenceClassification,AdamW, get_linear_schedule_with_warmup,Trainer, TrainingArguments
from transformers import DataCollator, DataCollatorForLanguageModeling,default_data_collator
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
import random
from textwrap import wrap
from datetime import datetime
from datasets import load_from_disk
from datasets import load_dataset
from datasets import Dataset
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from torch import nn

In [4]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
MODEL_NAME = "distilbert-base-uncased"
# max sequence length for each document/sentence sample
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE= 6.58e-5
WEIGHT_DECAY = 0.289
WARMUP_STEPS = 464
RANDOM_SEED=22


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
def set_seed(seed):
    """Set all seeds to make results reproducible (deterministic mode).
       When seed is None, disables deterministic mode.
    :param seed: an integer to your choosing
    """
    if seed is not None:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(seed)
        random.seed(seed)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  confusion_matrix = classification_report(labels, preds, digits=4,output_dict=True)
  return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'hate_f1': confusion_matrix["0"]["f1-score"],
        'hate_recall': confusion_matrix["0"]["recall"],
        'hate_precision': confusion_matrix["0"]["precision"],
        'offensive_f1': confusion_matrix["1"]["f1-score"],
        'offensive_recall': confusion_matrix["1"]["recall"],
        'offensive_precision': confusion_matrix["1"]["precision"],
        'normal_f1': confusion_matrix["2"]["f1-score"],
        'normal_recall': confusion_matrix["2"]["recall"],
        'normal_precision': confusion_matrix["2"]["precision"],    
  }




def model_init_two_layer_reinit():
  temp_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=3).to(device)
  temp_model = reinit_autoencoder_model(temp_model,2)
  return temp_model



def timestamp():
    dateTimeObj = datetime.now()
    timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")
    print(timestampStr)


def reinit_autoencoder_model(model, reinit_num_layers=0):
    """reinitialize autoencoder model layers"""

    if reinit_num_layers:
        for layer in model.distilbert.transformer.layer[-reinit_num_layers:]:
            for module in layer.modules():
                if isinstance(module, nn.Embedding):
                  if module.weight.requires_grad:
                    module.weight.data.normal_(mean=0.0, std=model.config.initializer_range)
                if isinstance(module, nn.Linear):
                  module.weight.data.normal_(mean=0.0, std=model.config.initializer_range)
                elif isinstance(module, nn.LayerNorm):
                  module.bias.data.zero_()
                  module.weight.data.fill_(1.0)
                if isinstance(module, nn.Linear) and module.bias is not None:
                  module.bias.data.zero_()

    return model



In [6]:
set_seed(RANDOM_SEED)

In [7]:
dataset_dfs = load_from_disk('/content/drive/MyDrive/Dissertation/datasets/hatetwit_'+str(1))

In [8]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results',          # output directory
    num_train_epochs=EPOCHS,              # total number of training epochs
    save_strategy ="epoch" ,
    per_device_train_batch_size=BATCH_SIZE,  # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,   # batch size for evaluation
    weight_decay= WEIGHT_DECAY,               # strength of weight decay
    learning_rate= LEARNING_RATE, 
    logging_dir='./disbert_hate_task/hyper/logs',     # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    evaluation_strategy="epoch",
    #eval_steps = 500     # evaluate each `logging_steps`
)

In [9]:
hyper_trainer = Trainer(
    model_init=model_init_two_layer_reinit,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset_dfs['train'],         # training dataset
    eval_dataset=dataset_dfs['validation'],          # evaluation dataset
    compute_metrics=compute_metrics     # the callback that computes metrics of interest
)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}

loading we

In [10]:
def hp_space_optuna(trial) :
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 3),
        "seed": trial.suggest_int("seed", 1, 40),
        "warmup_steps": trial.suggest_int("warmup_steps", 0, 500),
        "weight_decay": trial.suggest_float("weight_decay", 0, 0.3),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [ 16, 32, 64]),
    }

In [11]:
best_trial = hyper_trainer.hyperparameter_search(n_trials=40, direction="maximize", backend="optuna", hp_space=hp_space_optuna)

[I 2022-02-13 18:50:58,475] A new study created in memory with name: no-name-3b742680-6be4-406d-9109-d5ecf6ec790b
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinu

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.586600,0.544256,0.780425,0.735791,0.732764,0.740217,0.717090,0.753521,0.684018,0.857196,0.847834,0.866768,0.633086,0.619295,0.647505
2,0.497300,0.518703,0.793733,0.739827,0.759422,0.728901,0.730712,0.738431,0.723153,0.868852,0.902629,0.837513,0.619918,0.545643,0.717599
3,0.428900,0.510194,0.793518,0.748525,0.749122,0.750395,0.737345,0.776660,0.701818,0.867050,0.865605,0.868499,0.641180,0.608921,0.677047


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: attention_mask_bert, sentence, token_type_ids_bert, __index_level_0__, input_ids_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-0/checkpoint-1165
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-0/checkpoint-1165/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-0/checkpoint-1165/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: attention_mask_bert, sentence, token_type_ids_bert, __index_level_0__, input_ids_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Savin

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.632100,0.596592,0.757029,0.695887,0.713462,0.688712,0.689689,0.713280,0.667608,0.841616,0.871529,0.813688,0.556355,0.481328,0.659091
2,0.575600,0.566918,0.767976,0.713900,0.722245,0.709366,0.703922,0.722334,0.686424,0.849211,0.866346,0.832740,0.588568,0.539419,0.647572
3,0.564800,0.563479,0.769693,0.712281,0.725260,0.707900,0.709832,0.744467,0.678277,0.851879,0.873010,0.831746,0.575133,0.506224,0.665757


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: attention_mask_bert, sentence, token_type_ids_bert, __index_level_0__, input_ids_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-1/checkpoint-2330
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-1/checkpoint-2330/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-1/checkpoint-2330/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: attention_mask_bert, sentence, token_type_ids_bert, __index_level_0__, input_ids_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Savin

...

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: attention_mask_bert, sentence, token_type_ids_bert, __index_level_0__, input_ids_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
[I 2022-02-13 21:13:29,069] Trial 37 pruned. 
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.634200,0.541557,0.774415,0.736718,0.725973,0.751451,0.723424,0.773642,0.679329,0.848473,0.812662,0.887586,0.638256,0.668050,0.611006
2,0.472600,0.498773,0.803391,0.752557,0.771716,0.742720,0.762327,0.777666,0.747582,0.872857,0.904850,0.843049,0.622485,0.545643,0.724518
3,0.359900,0.519240,0.803821,0.761829,0.760504,0.765092,0.770704,0.809859,0.735160,0.872234,0.868197,0.876308,0.642549,0.617220,0.670045


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: attention_mask_bert, sentence, token_type_ids_bert, __index_level_0__, input_ids_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-38/checkpoint-583
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-38/checkpoint-583/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-38/checkpoint-583/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: attention_mask_bert, sentence, token_type_ids_bert, __index_level_0__, input_ids_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Savin

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.551700,0.521829,0.785362,0.744971,0.738410,0.758186,0.729888,0.816901,0.659626,0.859004,0.830063,0.890036,0.646022,0.627593,0.665567
2,0.418300,0.496331,0.798884,0.752729,0.754442,0.756197,0.756656,0.814889,0.706190,0.871776,0.869678,0.873884,0.629754,0.584025,0.683252


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: attention_mask_bert, sentence, token_type_ids_bert, __index_level_0__, input_ids_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-39/checkpoint-2330
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-39/checkpoint-2330/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-39/checkpoint-2330/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: attention_mask_bert, sentence, token_type_ids_bert, __index_level_0__, input_ids_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Sa

In [12]:
best_trial

BestRun(run_id='6', objective=9.96546190688163, hyperparameters={'learning_rate': 6.779448121064052e-05, 'num_train_epochs': 3, 'seed': 31, 'warmup_steps': 491, 'weight_decay': 0.05951904479543706, 'per_device_train_batch_size': 64})